In [4]:
from utils import Train_Model_and_Classify as TMC
import os


# 示例用法
training_dirs = [
    os.path.join('E:/Files', 'Acoustic Data', 'Ruminating'), 
    os.path.join('E:/Files', 'Acoustic Data', 'Gulp'),
    os.path.join('E:/Files', 'Acoustic Data', 'Breath')
] 
model_name = "svm_Ruminating_Gulp_Breath"
classifier_type = "svm_rbf"
test_audio_file = os.path.join('E:/Files', 'Acoustic Data', 'Test', '2min_new.wav')
segments_file = os.path.join('E:/Files', 'Acoustic Data', 'Test', '2min_new.segments')

TMC.audio_analysis_train_and_classify(training_dirs, mt_win =1, mt_step=1,s_win=0.1,s_step=0.05, model_name, classifier_type, test_audio_file, segments_file)


SyntaxError: positional argument follows keyword argument (1172051915.py, line 16)

In [2]:
from pyAudioAnalysis.audioSegmentation import mid_term_file_classification
help(mid_term_file_classification)

Help on function mid_term_file_classification in module pyAudioAnalysis.audioSegmentation:

mid_term_file_classification(input_file, model_name, model_type, plot_results=False, gt_file='')
    This function performs mid-term classification of an audio stream.
    Towards this end, supervised knowledge is used,
    i.e. a pre-trained classifier.
    ARGUMENTS:
        - input_file:        path of the input WAV file
        - model_name:        name of the classification model
        - model_type:        svm or knn depending on the classifier type
        - plot_results:      True if results are to be plotted using
                             matplotlib along with a set of statistics
    
    RETURNS:
          - segs:           a sequence of segment's endpoints: segs[i] is the
                            endpoint of the i-th segment (in seconds)
          - classes:        a sequence of class flags: class[i] is the
                            class ID of the i-th segment



E:\Python\Lib\site-packages\pyAudioAnalysis\..\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
